# OpenAI API

In [17]:
import json 
from openai import OpenAI

## Create chat completion

https://platform.openai.com/docs/api-reference/chat/create

### Default

In [2]:
client = OpenAI()

completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "developer", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Hello!"}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)


In [11]:
print(completion.to_json())

{
  "id": "chatcmpl-BKKltMmLBi5Up7OpaYjB12IE3chYM",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "Hello! How can I assist you today?",
        "refusal": null,
        "role": "assistant",
        "annotations": []
      }
    }
  ],
  "created": 1744186105,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_432e014d75",
  "usage": {
    "completion_tokens": 10,
    "prompt_tokens": 19,
    "total_tokens": 29,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


In [7]:
[x for x in dir(completion) if not x.startswith('_') and not callable(getattr(completion, x))]

['choices',
 'created',
 'id',
 'model',
 'model_computed_fields',
 'model_config',
 'model_extra',
 'model_fields',
 'model_fields_set',
 'object',
 'service_tier',
 'system_fingerprint',
 'usage']

In [12]:
print(completion.usage.to_json())

{
  "completion_tokens": 10,
  "prompt_tokens": 19,
  "total_tokens": 29,
  "completion_tokens_details": {
    "accepted_prediction_tokens": 0,
    "audio_tokens": 0,
    "reasoning_tokens": 0,
    "rejected_prediction_tokens": 0
  },
  "prompt_tokens_details": {
    "audio_tokens": 0,
    "cached_tokens": 0
  }
}


### Image input

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What's in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                    }
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0])


Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The image shows a scenic landscape with a wooden boardwalk path running through a grassy field. The sky is a clear blue with a few scattered clouds. There are trees and shrubs in the background, indicating a natural or park-like setting.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))


In [14]:
print(response.usage.to_json())

{
  "completion_tokens": 48,
  "prompt_tokens": 1117,
  "total_tokens": 1165,
  "completion_tokens_details": {
    "accepted_prediction_tokens": 0,
    "audio_tokens": 0,
    "reasoning_tokens": 0,
    "rejected_prediction_tokens": 0
  },
  "prompt_tokens_details": {
    "audio_tokens": 0,
    "cached_tokens": 0
  }
}


### Logprobs

In [ ]:
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": "Hello!"}
  ],
  logprobs=True,
  top_logprobs=2
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Hello! How can I assist you today?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[])


In [16]:
print(completion.choices[0].logprobs)


ChoiceLogprobs(content=[ChatCompletionTokenLogprob(token='Hello', bytes=[72, 101, 108, 108, 111], logprob=-0.25193455815315247, top_logprobs=[TopLogprob(token='Hello', bytes=[72, 101, 108, 108, 111], logprob=-0.25193455815315247), TopLogprob(token='Hi', bytes=[72, 105], logprob=-1.50193452835083)]), ChatCompletionTokenLogprob(token='!', bytes=[33], logprob=-3.7697225252486533e-06, top_logprobs=[TopLogprob(token='!', bytes=[33], logprob=-3.7697225252486533e-06), TopLogprob(token=' there', bytes=[32, 116, 104, 101, 114, 101], logprob=-12.500003814697266)]), ChatCompletionTokenLogprob(token=' How', bytes=[32, 72, 111, 119], logprob=-1.1472419600977446e-06, top_logprobs=[TopLogprob(token=' How', bytes=[32, 72, 111, 119], logprob=-1.1472419600977446e-06), TopLogprob(token=' What', bytes=[32, 87, 104, 97, 116], logprob=-13.750000953674316)]), ChatCompletionTokenLogprob(token=' can', bytes=[32, 99, 97, 110], logprob=-4.246537173457909e-06, top_logprobs=[TopLogprob(token=' can', bytes=[32, 99,

### Structured output

https://platform.openai.com/docs/guides/structured-outputs?api-mode=chat

In [ ]:
from pydantic import BaseModel

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

try:
    completion = client.beta.chat.completions.parse(
        model="gpt-4o-2024-08-06",
        messages=[
            {"role": "system", "content": "Extract the event information."},
            {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
        ],
        response_format=CalendarEvent,
    )

    response_message = completion.choices[0].message 
    event = response_message.parsed

    # If the model refuses to respond, you will get a refusal message
    if (response_message.refusal):
        print(response_message.refusal)
    else:
        print(response_message.parsed)
except:
    pass

In [24]:
print(event)

name='Science Fair' date='Friday' participants=['Alice', 'Bob']


In [27]:
completion.usage

CompletionUsage(completion_tokens=18, prompt_tokens=92, total_tokens=110, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

### Structured output using `instructor`

In [ ]:
import instructor

# Patch the OpenAI client
        
client_instructor = instructor.from_openai(client)

user_info, completion = client_instructor.chat.completions.create_with_completion(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
        ],
    response_model=CalendarEvent,
    )

In [39]:
print(user_info)

name='Science Fair' date='Friday' participants=['Alice', 'Bob']


In [38]:
print(completion)

ParsedChatCompletion[CalendarEvent](id='chatcmpl-BKLhDHMc715WKiW5HF8qX1RJhLZJR', choices=[ParsedChoice[CalendarEvent](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[CalendarEvent](content='{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None, parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])))], created=1744189659, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_92f14e8683', usage=CompletionUsage(completion_tokens=18, prompt_tokens=92, total_tokens=110, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [40]:
print(completion.usage)

CompletionUsage(completion_tokens=18, prompt_tokens=92, total_tokens=110, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))


### Structured output using `trustcall`

### Structured output using `outlines`

## Create a model response

+ https://platform.openai.com/docs/api-reference/responses/create
+ https://platform.openai.com/docs/guides/responses-vs-chat-completions

### Text input

In [ ]:
response = client.responses.create(
  model="gpt-4o",
  # instructions="You are a famous novels writer in the romanticism age.",
  # input="Tell me a three sentence bedtime story about a unicorn."
  # Alternatively, you can use the following format for the input parameter:
  input=[
        {"role": "system", "content": "You are a famous novels writer in the romanticism age."},
        {"role": "user", "content": "Tell me a three sentence bedtime story about a unicorn."}
    ],
)

print(response)


Response(id='resp_67f63671d57c8191b5a4b4cfaca7a4600e8def67552d7c30', created_at=1744189041.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[ResponseOutputMessage(id='msg_67f63672268881919ad1ac39b90543040e8def67552d7c30', content=[ResponseOutputText(annotations=[], text="In a moonlit forest where whispering willows swayed, a unicorn with a mane of stardust roamed in search of dreams spun from silver. One night, she met a lonely child who wished for friendship, and with a gentle touch of her horn, turned the child's tears into a cascade of shimmering fireflies. Together, they danced beneath the night sky, their laughter echoing through eternity, leaving traces of magic wherever they wandered.", type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasonin

In [5]:
print(response.to_json())

{
  "id": "resp_67f62f3b29e88191a4b7fdf8664f65f0074a3484430c8165",
  "created_at": 1744187195.0,
  "error": null,
  "incomplete_details": null,
  "instructions": null,
  "metadata": {},
  "model": "gpt-4o-2024-08-06",
  "object": "response",
  "output": [
    {
      "id": "msg_67f62f3be3fc8191a6e903c53b8df5e5074a3484430c8165",
      "content": [
        {
          "annotations": [],
          "text": "In a serene glade hidden deep within the ancient Whispering Woods, lived a gentle unicorn named Luna, whose shimmering horn was said to light up with the dreams of the world. One starry night, Luna noticed a curious flicker in the sky, revealing a tiny lost star that had tumbled from its celestial home. With a tender touch of her glowing horn, Luna lifted the star back to the heavens, ensuring that it would continue to twinkle brightly for all dreamers below.",
          "type": "output_text"
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "messa

In [11]:
[x for x in dir(response) if not x.startswith('_') and not callable(getattr(response, x))]

['created_at',
 'error',
 'id',
 'incomplete_details',
 'instructions',
 'max_output_tokens',
 'metadata',
 'model',
 'model_computed_fields',
 'model_config',
 'model_extra',
 'model_fields',
 'model_fields_set',
 'object',
 'output',
 'output_text',
 'parallel_tool_calls',
 'previous_response_id',
 'reasoning',
 'status',
 'temperature',
 'text',
 'tool_choice',
 'tools',
 'top_p',
 'truncation',
 'usage',
 'user']

In [14]:
print(response.usage.to_json())

{
  "input_tokens": 34,
  "input_tokens_details": {
    "cached_tokens": 0
  },
  "output_tokens": 99,
  "output_tokens_details": {
    "reasoning_tokens": 0
  },
  "total_tokens": 133
}


### Image input

In [8]:
response = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": [
                { "type": "input_text", "text": "what is in this image?" },
                {
                    "type": "input_image",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg"
                }
            ]
        }
    ]
)

print(response)


Response(id='resp_67f6314acbb48191bce232e9f9b781460d4342fbe4f841c8', created_at=1744187722.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-2024-08-06', object='response', output=[ResponseOutputMessage(id='msg_67f6314c5c288191bf62d60ade3d2c6e0d4342fbe4f841c8', content=[ResponseOutputText(annotations=[], text='The image shows a scenic landscape with a wooden boardwalk path leading through a grassy field. The sky is blue with some clouds, creating a peaceful and open atmosphere.', type='output_text')], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, generate_summary=None), status='completed', text=ResponseTextConfig(format=ResponseFormatText(type='text')), truncation='disabled', usage=ResponseUsage(input_tokens=1118, input_tokens_details=InputTokensDetails(cached_tokens=0), outp

In [9]:
print(response.usage.to_json())

{
  "input_tokens": 1118,
  "input_tokens_details": {
    "cached_tokens": 0
  },
  "output_tokens": 34,
  "output_tokens_details": {
    "reasoning_tokens": 0
  },
  "total_tokens": 1152
}


### Logprobs

NO TIENE

### Structured output

https://platform.openai.com/docs/guides/structured-outputs

In [ ]:
try:
    response = client.responses.create(
        model="gpt-4o-2024-08-06",
        input=[
            {"role": "system", "content": "Extract the event information."},
            {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."}
        ],
        text={
            "format": {
                "type": "json_schema",
                "name": "calendar_event",
                "schema": {
                    "type": "object",
                    "properties": {
                        "name": {
                            "type": "string"
                        },
                        "date": {
                            "type": "string"
                        },
                        "participants": {
                            "type": "array", 
                            "items": {
                                "type": "string"
                            }
                        },
                    },
                    "required": ["name", "date", "participants"],
                    "additionalProperties": False
                },
                "strict": True
            }
        }
    )
except Exception as e:
    # handle errors like finish_reason, refusal, content_filter, etc.
    pass

In [25]:
try:
    response = client.responses.create(
        model="gpt-4o-2024-08-06",
        input=[
            {"role": "system", "content": "Extract the event information."},
            {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."}
        ],
        response_format=CalendarEvent
    )
except Exception as e:
    # handle errors like finish_reason, refusal, content_filter, etc.
    pass

In [26]:
event = json.loads(response.output_text)
print(event)

{'name': 'Science Fair', 'date': 'Friday', 'participants': ['Alice', 'Bob']}


### Structured output using `instructor`

In [42]:
import instructor

# Patch the OpenAI client
        
client_instructor = instructor.from_openai(client)

# user_info, completion = client_instructor.chat.completions.create_with_completion(
res = client_instructor.responses(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": "Extract the event information."},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
        ],
    response_model=CalendarEvent,
    )

TypeError: 'Responses' object is not callable

In [ ]:
print(user_info)

name='Science Fair' date='Friday' participants=['Alice', 'Bob']


In [ ]:
print(completion)

ParsedChatCompletion[CalendarEvent](id='chatcmpl-BKLhDHMc715WKiW5HF8qX1RJhLZJR', choices=[ParsedChoice[CalendarEvent](finish_reason='stop', index=0, logprobs=None, message=ParsedChatCompletionMessage[CalendarEvent](content='{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None, parsed=CalendarEvent(name='Science Fair', date='Friday', participants=['Alice', 'Bob'])))], created=1744189659, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_92f14e8683', usage=CompletionUsage(completion_tokens=18, prompt_tokens=92, total_tokens=110, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))


In [ ]:
print(completion.usage)

CompletionUsage(completion_tokens=18, prompt_tokens=92, total_tokens=110, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))


### Structured output using `trustcall`

### Structured output using `outlines`

# Langchain API

# Outlines API